### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [3]:
# read the data into pandas dataframes
MN_T00_GF_126a = pd.read_csv("lca/cleaned/126_MN_T00_GF_lca.csv")

# delete any Metazoa hits because they're trypsin
MN_T00_GF_126b = MN_T00_GF_126a[MN_T00_GF_126a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MN_T00_GF_126 = MN_T00_GF_126b[MN_T00_GF_126b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MN_T00_GF_126))

MN_T00_GF_126.head(6)

# of phylum peptides =  26


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
3,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,VVTADLPR,Chloroflexi,Bacteria,NaN,NaN,NaN,Chloroflexi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,VGHWLPR,Streptoalloteichus hindustanus,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,Streptoalloteichus,NaN,NaN,NaN,Streptoalloteichus hindustanus,NaN,NaN,NaN,NaN,NaN
29,VATVPSLR,Herpotrichiellaceae,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Pezizomycotina,NaN,Eurotiomycetes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,LSVNLNSK,Listeria monocytogenes,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Bacilli,...,Listeria,NaN,NaN,NaN,Listeria monocytogenes,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [6]:
MN_T00_GF_126_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/126_MN_T00_GF_DN50_ILnaafs.csv")
MN_T00_GF_126_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/126B_MN_T00_GF_DN50_ILnaafs.csv")
#MN_T00_GF_126_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/126C_MN_T00_GF_DN50_ILnaafs.csv")


frames = [MN_T00_GF_126_NAAFa, MN_T00_GF_126_NAAFb]

MN_T00_GF_126_NAAF = pd.concat(frames, sort=False)

MN_T00_GF_126_NAAF.set_index('stripped_peptide')
MN_T00_GF_126_NAAF = MN_T00_GF_126_NAAF.loc[:, ~MN_T00_GF_126_NAAF.columns.str.contains('^Unnamed')]

MN_T00_GF_126_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MN_T00_GF_126_NAAF))

print('column names:', MN_T00_GF_126_NAAF.columns)

MN_T00_GF_126_NAAF.head()

# of total peptides =  322
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,174000.0,1.338462e+04
1,LSSPATLNSR,61300000.0,6.130000e+06
2,LSSPATLNSR,61300000.0,6.130000e+06
3,LSSPATLNSR,61300000.0,6.130000e+06
4,LSSPATLDSR,470000.0,4.700000e+04


In [7]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MN_T00_GF_126["peptide"], MN_T00_GF_126_NAAF["peptide"])

print(over)

['APVLSMR' 'LATVLSPR' 'LSVNLNSK' 'TMLPVPR' 'TTTVSLPR' 'VATVPSLR'
 'VATVSPLR' 'VGHWLPR' 'VLGQNEAVNAVSNALR' 'VVEVSLPR' 'VVTADLPR' 'VVTVDVPR']


In [8]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MN_T00_GF_126.set_index('peptide', inplace=True)
MN_T00_GF_126_NAAF.set_index('peptide', inplace=True)

MN_T00_GF_126_Phy = MN_T00_GF_126.join(MN_T00_GF_126_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MN_T00_GF_126_Phy))

MN_T00_GF_126_Phy.head()

# of total phylum-level peptides =  39


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
ALTVVDPR,Opitutae,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,Opitutae,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APVLSMR,Brevundimonas,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46700.0,6671.428571
DAVCPTLR,Aspergillus oryzae,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Pezizomycotina,NaN,Eurotiomycetes,Eurotiomycetidae,...,NaN,NaN,Aspergillus oryzae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GELTGLNK,Elapid 1 orthobornavirus,Viruses,Orthornavirae,NaN,NaN,Negarnaviricota,Haploviricotina,NaN,Monjiviricetes,NaN,...,NaN,NaN,Elapid 1 orthobornavirus,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GVGLDGQAVLLK,Candidatus Methylopumilus turicensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Betaproteobacteria,NaN,...,NaN,NaN,Candidatus Methylopumilus turicensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# write to a csv

MN_T00_GF_126_Phy.to_csv("lca/NAAF/MN_T00_GF_126_Phy_naaf.csv")